Some sources:

* Gensim LDA: https://radimrehurek.com/gensim/models/ldamodel.html
* Misc clustering with Python: http://brandonrose.org/clustering
* Scikit LDA: http://scikit-learn.org/0.16/modules/generated/sklearn.lda.LDA.html
* Scikit NMF: http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html
* WMD in Python: http://vene.ro/blog/word-movers-distance-in-python.html
* Original WMD paper: http://jmlr.org/proceedings/papers/v37/kusnerb15.pdf
* Scikit Affinity propagation: http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html

# Greek (TLG)

In [2]:
import os
import time

In [1]:
from cltk.corpus.greek.tlg.parse_tlg_indices import get_epithet_index
from cltk.corpus.greek.tlg.parse_tlg_indices import get_epithets
from cltk.corpus.greek.tlg.parse_tlg_indices import select_authors_by_epithet
from cltk.corpus.greek.tlg.parse_tlg_indices import get_epithet_of_author
from cltk.corpus.greek.tlg.parse_tlg_indices import get_id_author
from cltk.stop.greek.stops import STOPS_LIST as greek_stops
from cltk.tokenize.word import nltk_tokenize_words

from greek_accentuation.characters import base

import pandas

from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
def stream_lemmatized_files(corpus_dir):
    # return all docs in a dir
    user_dir = os.path.expanduser('~/cltk_data/user_data/' + corpus_dir)
    files = os.listdir(user_dir)

    for file in files:
        filepath = os.path.join(user_dir, file)
        with open(filepath) as fo:
            yield fo.read()

In [11]:
def map_file_order_to_name(corpus_dir):
    # read contents of dir, order and their tlg name
    user_dir = os.path.expanduser('~/cltk_data/user_data/' + corpus_dir)
    files = os.listdir(user_dir)
    
    map_id_author = get_id_author()  # {…, '1648': 'Pyrgion Hist.', '4017': 'Syrianus Phil.', …}

    map_count_name = {}
    for count, file in enumerate(files):
        file_id = str(file[3:-4])
        name = map_id_author[file_id]
        map_count_name[count] = name
    
    return map_count_name

In [14]:
map_file_count_name = map_file_order_to_name('tlg_lemmatized_no_accents_no_stops')

In [15]:
t0 = time.time()

data_samples = []
for text in stream_lemmatized_files('tlg_lemmatized_no_accents_no_stops'):
    data_samples.append(text)

print('Time to collect texts: {}'.format(time.time() - t0))
print('Number of texts:', len(data_samples))

Time to collect texts: 4.774492979049683
Number of texts: 1823


In [117]:
# view all epithets:
get_epithets()

['Alchemistae',
 'Apologetici',
 'Astrologici',
 'Astronomici',
 'Atticistae',
 'Biographi',
 'Bucolici',
 'Choliambographi',
 'Chronographi',
 'Comici',
 'Doxographi',
 'Elegiaci',
 'Epici/-ae',
 'Epigrammatici/-ae',
 'Epistolographi',
 'Geographi',
 'Geometri',
 'Gnomici',
 'Gnostici',
 'Grammatici',
 'Hagiographi',
 'Historici/-ae',
 'Hymnographi',
 'Iambici',
 'Lexicographi',
 'Lyrici/-ae',
 'Mathematici',
 'Mechanici',
 'Medici',
 'Mimographi',
 'Musici',
 'Mythographi',
 'Nomographi',
 'Onirocritici',
 'Oratores',
 'Paradoxographi',
 'Parodii',
 'Paroemiographi',
 'Periegetae',
 'Philologi',
 'Philosophici/-ae',
 'Poetae',
 'Poetae Didactici',
 'Poetae Medici',
 'Poetae Philosophi',
 'Polyhistorici',
 'Rhetorici',
 'Scriptores Ecclesiastici',
 'Scriptores Erotici',
 'Scriptores Fabularum',
 'Scriptores Rerum Naturalium',
 'Sophistae',
 'Tactici',
 'Theologici',
 'Tragici']

In [16]:
# tf-idf features
n_samples = 2000
n_features = 1000  # TODO: increase
n_topics = len(get_epithets())
n_top_words = 20

tfidf_vectorizer = TfidfVectorizer(max_df=1.0, 
                                   min_df=1,
                                   max_features=n_features,
                                   stop_words=None)
t0 = time.time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print('Time to extract tf-idf features: {} secs.'.format(time.time() - t0))

Time to extract tf-idf features: 73.5945999622345 secs.


In [17]:
print("Fitting the NMF model with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time.time()
nmf = NMF(n_components=n_topics, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print('Fit NMF model in {}'.format(time.time() - t0))

Fitting the NMF model with tf-idf features, n_samples=2000 and n_features=1000...
Fit NMF model in 7.5813329219818115


In [18]:
def print_top_words(model, feature_names, n_top_words):
    for topic_id, topic in enumerate(model.components_):
        print('Topic #{}:'.format(int(topic_id)))
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2))
              +' | ' for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print()

In [19]:
print("Topics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

print_top_words(nmf, tfidf_feature_names, n_top_words)

Topics in NMF model:
Topic #0:
ειμι 2.55 | ου 2.46 | τος 1.31 | αυ 1.07 | εχω 0.97 | τας 0.68 | αλλ 0.52 | δεω1 0.43 | ποιεω 0.43 | εγω 0.42 | ωσπερ 0.42 | τουτων 0.41 | εαυτου 0.39 | πα 0.37 | οσος 0.33 | καθ 0.33 | ευ 0.33 | ος 0.33 | ουδεν 0.32 | δι 0.31 | 

Topic #1:
αυ 3.52 | τος 3.44 | αυτον 0.79 | αυτους 0.32 | αυτην 0.28 | τας 0.24 | τοτε 0.17 | εγενετο 0.15 | εαυτου 0.15 | εκει 0.15 | πολιν 0.15 | υπ 0.14 | μετ 0.14 | πολεως 0.14 | τουτου 0.13 | ηως 0.13 | αδελφος 0.13 | πολυς 0.13 | ερχομαι 0.13 | γυνη 0.12 | 

Topic #2:
εγω 4.05 | εμεω 0.24 | σε 0.23 | εμοι 0.17 | ειδον 0.16 | εμε 0.15 | συ 0.14 | παρ 0.14 | νυ 0.13 | οπως 0.12 | ταυτ 0.12 | οιδα 0.11 | ερχομαι 0.1 | πως 0.09 | ουδ 0.09 | σευω 0.09 | ποθεν 0.09 | ειπε 0.09 | καγω 0.09 | χρηματα 0.08 | 

Topic #3:
φημι 3.37 | αυτον 0.42 | ειμι 0.35 | φησι 0.24 | γενεσθαι 0.23 | λεγει 0.23 | φασι 0.21 | ιστορεω 0.19 | καθα 0.19 | φησιν 0.17 | καλεω 0.17 | αθηνη 0.16 | υστερος 0.16 | ποτε 0.15 | ομηρος 0.15 | ον 0.14 | λεγεται 

In [20]:
tfidf.shape

(1823, 1000)

In [21]:
doc_topic_distrib = nmf.transform(tfidf)  # numpy.ndarray

In [22]:
doc_topic_distrib.shape

(1823, 55)

In [71]:
df = pandas.DataFrame(doc_topic_distrib)

In [72]:
df = df.rename(map_file_count_name)

Questions:

* to what topic does each author most belong? (and how to determine cutoff?)
* what authors most exemplify a topic?

In [73]:
df

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
Apollonius Rhodius Epic.,0.008242,0.036524,0.031082,0.007385,0.030441,0.276695,0.000000,0.035696,0.000000,0.022861,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.027332,0.000000,0.000000,0.016152,0.000000
Theognis Eleg.,0.047067,0.000000,0.046855,0.000000,0.051642,0.134257,0.054331,0.023679,0.000000,0.000000,...,0.000000,0.008999,0.003188,0.000000,0.000000,0.013205,0.000000,0.000000,0.000000,0.000000
Thucydides Hist.,0.101537,0.065956,0.011589,0.000000,0.022067,0.021074,0.001556,0.017924,0.001924,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.006843,0.000000,0.000000,0.000000,0.000000
Diogenes Laertius Biogr.,0.113660,0.046509,0.007916,0.074166,0.002784,0.000000,0.000000,0.033737,0.000000,0.006540,...,0.000000,0.002134,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Theocritus Bucol.,0.054391,0.015502,0.067163,0.000000,0.027391,0.166300,0.020212,0.033731,0.000000,0.000000,...,0.000000,0.000000,0.008156,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Euripides Trag.,0.064044,0.000000,0.061053,0.000000,0.056361,0.095156,0.026170,0.049990,0.002007,0.000000,...,0.000000,0.000000,0.005795,0.000000,0.000000,0.010141,0.000000,0.000000,0.000000,0.000000
Plutarchus Biogr. et Phil.,0.158096,0.029795,0.001092,0.015922,0.019112,0.014260,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Athenaeus Soph.,0.123865,0.029537,0.014366,0.080690,0.009208,0.023742,0.018648,0.020011,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Sappho Lyr.,0.024411,0.000000,0.108845,0.001343,0.033773,0.122476,0.024436,0.024487,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010710,0.000000,0.000000,0.000000,0.000000
Isocrates Orat.,0.129990,0.033435,0.042834,0.000000,0.014657,0.000000,0.012045,0.000000,0.001898,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.017119,0.000000,0.000000,0.000000,0.000000


In [74]:
# for each topic (col), which author (row) has the highest value?
# TODO: get top 5 authors

for count in range(n_topics):
    print('Top author of topic {0}: {1}'.format(count, df[count].idxmax()))

Top author of topic 0: Joannes Rhet.
Top author of topic 1: Cephalion Hist. et Rhet.
Top author of topic 2: Pamphilus Poeta
Top author of topic 3: Heraclides Phil.
Top author of topic 4: Eubulides Comic.
Top author of topic 5: Quintus Epic.
Top author of topic 6: Stratonicus Poeta
Top author of topic 7: Xenocles Trag.
Top author of topic 8: Capito Hist.
Top author of topic 9: Dionysius Thrax Gramm.
Top author of topic 10: Lexica Syntactica
Top author of topic 11: Melanthius Trag.
Top author of topic 12: Canon Librorum
Top author of topic 13: Euclides Geom.
Top author of topic 14: Chronographiae Anonymae
Top author of topic 15: Diogenes Phil.
Top author of topic 16: Moeris Attic.
Top author of topic 17: Mace(donius) Lyr.
Top author of topic 18: Autocrates Hist.
Top author of topic 19: [Alexarchus] Hist.
Top author of topic 20: Demetrius Comic.
Top author of topic 21: Choerilus Trag.
Top author of topic 22: Fragmentum Stoicum
Top author of topic 23: Acta Phileae
Top author of topic 24: L

In [105]:
# Now, transpose df and get top topic of each author
# for each topic (col), which author (row) has the highest value?
# TODO: get top 5 authors
df_t = df.T

In [108]:
df_t

,Apollonius Rhodius Epic.,Theognis Eleg.,Thucydides Hist.,Diogenes Laertius Biogr.,Theocritus Bucol.,Euripides Trag.,Plutarchus Biogr. et Phil.,Athenaeus Soph.,Sappho Lyr.,Isocrates Orat.,...,Macarius Chrysocephalus Paroemiogr.,Michael Apostolius Paroemiogr.,"Suda, Suidas",Ignatius Biogr. et Poeta,Arsenius Paroemiogr.,Stephanus Phil.,Stephanus Gramm.,Stephanus Alchem.,Joannes Tzetzes Gramm. et Poeta,Thomas Magister Philol.
0,0.008242,0.047067,0.101537,0.113660,0.054391,0.064044,0.158096,0.123865,0.024411,0.129990,...,0.113071,0.136326,0.119459,0.079524,0.129651,0.091940,0.133484,0.088006,0.106135,0.105958
1,0.036524,0.000000,0.065956,0.046509,0.015502,0.000000,0.029795,0.029537,0.000000,0.033435,...,0.000000,0.041658,0.041848,0.064819,0.000000,0.000000,0.021613,0.058973,0.037904,0.014897
2,0.031082,0.046855,0.011589,0.007916,0.067163,0.061053,0.001092,0.014366,0.108845,0.042834,...,0.000000,0.000000,0.003493,0.016902,0.009843,0.000000,0.007494,0.008364,0.035237,0.009422
3,0.007385,0.000000,0.000000,0.074166,0.000000,0.000000,0.015922,0.080690,0.001343,0.000000,...,0.000000,0.014324,0.013797,0.000000,0.000000,0.000000,0.000000,0.000000,0.027534,0.000000
4,0.030441,0.051642,0.022067,0.002784,0.027391,0.056361,0.019112,0.009208,0.033773,0.014657,...,0.022974,0.019184,0.002021,0.041783,0.029201,0.055901,0.025907,0.000000,0.000000,0.067595
5,0.276695,0.134257,0.021074,0.000000,0.166300,0.095156,0.014260,0.023742,0.122476,0.000000,...,0.042272,0.030915,0.033264,0.006469,0.058484,0.000000,0.000000,0.000000,0.010836,0.000388
6,0.000000,0.054331,0.001556,0.000000,0.020212,0.026170,0.000000,0.018648,0.024436,0.012045,...,0.006544,0.010691,0.004577,0.000000,0.007182,0.002109,0.002197,0.008066,0.011759,0.004061
7,0.035696,0.023679,0.017924,0.033737,0.033731,0.049990,0.000000,0.020011,0.024487,0.000000,...,0.000000,0.011124,0.021885,0.000000,0.020511,0.065010,0.019407,0.025494,0.012284,0.000000
8,0.000000,0.000000,0.001924,0.000000,0.000000,0.002007,0.000000,0.000000,0.000000,0.001898,...,0.000000,0.000604,0.011667,0.000000,0.000000,0.000000,0.000000,0.000000,0.001467,0.000000
9,0.022861,0.000000,0.000000,0.006540,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.008514,0.000000,0.000000,0.039004,0.041932,0.000000,0.000000,0.028477


In [120]:
curr_name = 'Dionysius Perieg.'
for _id, name in get_id_author().items():
    if curr_name == name:
        print(_id)

TypeError: get_epithet_of_author() missing 1 required positional argument: '_id'

In [122]:
for curr_name in df_t.columns:
    print(curr_name)
    print('    NMF topic:', df_t[curr_name].idxmax())
    for _id, name in get_id_author().items():
        if curr_name == name:
            print('    Traditional epithet:', get_epithet_of_author(_id))
    print()

Apollonius Rhodius Epic.
    NMF topic: 5
    Traditional epithet: Epici/-ae

Theognis Eleg.
    NMF topic: 5
    Traditional epithet: Elegiaci

Thucydides Hist.
    NMF topic: 0
    Traditional epithet: Historici/-ae

Diogenes Laertius Biogr.
    NMF topic: 12
    Traditional epithet: Biographi

Theocritus Bucol.
    NMF topic: 5
    Traditional epithet: Bucolici

Euripides Trag.
    NMF topic: 5
    Traditional epithet: Tragici

Plutarchus Biogr. et Phil.
    NMF topic: 0
    Traditional epithet: Philosophici/-ae

Athenaeus Soph.
    NMF topic: 0
    Traditional epithet: Sophistae

Sappho Lyr.
    NMF topic: 30
    Traditional epithet: Lyrici/-ae

Isocrates Orat.
    NMF topic: 0
    Traditional epithet: Oratores

Sophocles Trag.
    NMF topic: 5
    Traditional epithet: Tragici

Homerus Epic., Homer
    NMF topic: 5
    Traditional epithet: Epici/-ae

Hymni Homerici, Homeric Hymns
    NMF topic: 5
    Traditional epithet: None

Demosthenes Orat.
    NMF topic: 0
    Traditional epit

In [75]:
# also do this all w/ countvectorizer?

In [76]:
# http://scikit-learn.org/stable/modules/clustering.html

In [79]:
dataset_array = df.values
print(dataset_array.dtype)  # kmeans needs to be homogeneous data type (here, float64)
print(dataset_array)

[[ 0.00824198  0.03652398  0.03108222 ...,  0.          0.01615235  0.        ]
 [ 0.04706742  0.          0.04685514 ...,  0.          0.          0.        ]
 [ 0.10153668  0.06595567  0.0115893  ...,  0.          0.          0.        ]
 ..., 
 [ 0.0880064   0.05897268  0.00836376 ...,  0.          0.          0.        ]
 [ 0.10613521  0.03790386  0.03523662 ...,  0.          0.          0.        ]
 [ 0.10595783  0.01489671  0.00942192 ...,  0.          0.          0.        ]]


In [80]:
# do I need to normalize
# sklearn.preprocessing.StandardScaler
from sklearn import preprocessing

In [81]:
# http://scikit-learn.org/stable/modules/preprocessing.html
# first load scaler and train on given data set
scaler = preprocessing.StandardScaler().fit(df)

In [83]:
scaler.mean_

array([ 0.05019921,  0.0255092 ,  0.01397297,  0.01327393,  0.01495358,
        0.01682819,  0.0106435 ,  0.01827097,  0.00683908,  0.00642039,
        0.01236455,  0.0085962 ,  0.00649821,  0.00558679,  0.00432717,
        0.00722202,  0.00215602,  0.00646187,  0.0057925 ,  0.00758634,
        0.00484418,  0.00656875,  0.00489019,  0.00575802,  0.00585413,
        0.00415333,  0.00372114,  0.0057934 ,  0.00227336,  0.0064333 ,
        0.00536679,  0.00394071,  0.0054923 ,  0.00498952,  0.00439297,
        0.00433457,  0.00455127,  0.00448185,  0.00398487,  0.0028697 ,
        0.00211415,  0.00439907,  0.00478673,  0.0031645 ,  0.00697561,
        0.0043648 ,  0.00606566,  0.00472907,  0.00291753,  0.01367439,
        0.00459177,  0.00421821,  0.00297059,  0.00310003,  0.00268476])

In [84]:
scaler.scale_

array([ 0.04603533,  0.03202729,  0.02586347,  0.02606835,  0.02198726,
        0.03800937,  0.02016901,  0.02141974,  0.03082646,  0.02172892,
        0.02452027,  0.01946246,  0.02956141,  0.03384907,  0.02631689,
        0.02008212,  0.02062244,  0.0179572 ,  0.02298408,  0.0250701 ,
        0.01600548,  0.01951937,  0.01934805,  0.02031044,  0.02704938,
        0.02358621,  0.01857639,  0.0184143 ,  0.02217991,  0.01903844,
        0.02314547,  0.01732269,  0.01553694,  0.0171219 ,  0.01732228,
        0.01877199,  0.01768333,  0.02566982,  0.02052266,  0.02259618,
        0.027696  ,  0.02023981,  0.01819136,  0.01867748,  0.02453883,
        0.02552122,  0.02422061,  0.01545062,  0.02224687,  0.03765047,
        0.01538947,  0.01859583,  0.02432938,  0.01869078,  0.02077123])

In [91]:
# actually do normalization; can be reused for eg a training set
df_scaled = pandas.DataFrame(scaler.transform(df))

In [92]:
from sklearn import cluster

In [99]:
# Convert DataFrame to matrix
matrix = df_scaled.as_matrix()

km = cluster.KMeans(n_clusters=n_topics)
km.fit(matrix)

# Get cluster assignment labels
labels = km.labels_

# Format results as a DataFrame
df_clusters = pandas.DataFrame([df_scaled.index, labels]).T  # add author names to the 0 col

In [100]:
df_clusters

,0,1
0,0,45
1,1,45
2,2,6
3,3,28
4,4,45
5,5,47
6,6,6
7,7,28
8,8,49
9,9,2


In [98]:
import matplotlib.pyplot as plt

In [104]:
df_clusters[1]

0       45
1       45
2        6
3       28
4       45
5       47
6        6
7       28
8       49
9        2
10       1
11      45
12      45
13      47
14      42
15       6
16      47
17      51
18      47
19      45
20      45
21      45
22      45
23      47
24      47
25      47
26      47
27      47
28       8
29       6
        ..
1793    54
1794     6
1795    38
1796    54
1797     6
1798    54
1799    54
1800    54
1801    54
1802     6
1803     6
1804    54
1805    51
1806    45
1807     8
1808    45
1809     2
1810    26
1811     6
1812     6
1813     6
1814     6
1815    54
1816     8
1817     6
1818    26
1819     6
1820    37
1821     2
1822    54
Name: 1, dtype: int64